<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/sqlite3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [199]:
import sqlite3
from tabulate import tabulate

db = 'site.db'

def show_tables():
  tables = ['users', 'chats', 'messages']
  def show(tablename):
    try:
      with sqlite3.connect(db) as connection:
        cursor = connection.cursor()
        cursor.execute(f'SELECT * FROM {tablename}')
        results = cursor.fetchall()
        headers = []
        for i in range(len(cursor.description)):
          headers.append(cursor.description[i][0])
        print(tablename)
        print(tabulate(results, headers=headers, tablefmt='grid', stralign='center'))
        print("\n")
    except:
      print("Empty")

  for table in tables:
    show(table)

show_tables()

users
+------+------------+------------+
|   id |  username  |   password |
+======+============+============+
|    1 |    Ivan    |      12345 |
+------+------------+------------+
|    2 |    Mike    |      12345 |
+------+------------+------------+


chats
+------+------------+-----------+
|   id |  chatname  |   user_id |
+======+============+===========+
|    1 | ivans_chat |         1 |
+------+------------+-----------+
|    2 | ivans_chat |         2 |
+------+------------+-----------+
|    3 | mikes_chat |         1 |
+------+------------+-----------+
|    4 | mikes_chat |         2 |
+------+------------+-----------+


messages
+------+-----------+-----------+-----------+---------------------+
|   id |   chat_id |   user_id |  message  |     created_at      |
+======+===========+===========+===========+=====================+
|    1 |         1 |         1 |   hello   | 2023-11-12 02:46:18 |
+------+-----------+-----------+-----------+---------------------+
|    2 |         2 | 

In [198]:
def create_db():
  with sqlite3.connect(db) as connection:
    cursor = connection.cursor()
    cursor.execute("""CREATE TABLE IF NOT EXISTS users (
                      id INTEGER PRIMARY KEY AUTOINCREMENT,
                      username VARCHAR(255) NOT NULL,
                      password VARCHAR(255) NOT NULL
                    );""")

    cursor.execute("""CREATE TABLE IF NOT EXISTS chats (
                      id INTEGER PRIMARY KEY AUTOINCREMENT,
                      chatname VARCHAR(255) NOT NULL,
                      user_id INTEGER NOT NULL,
                      FOREIGN KEY (user_id) REFERENCES users (id) ON DELETE CASCADE
                    );""")

    cursor.execute("""CREATE TABLE IF NOT EXISTS messages (
                      id INTEGER PRIMARY KEY AUTOINCREMENT,
                      chat_id INTEGER NOT NULL,
                      user_id INTEGER NOT NULL,
                      message TEXT NOT NULL,
                      created_at DATETIME NOT NULL,
                      FOREIGN KEY (chat_id) REFERENCES chats (id) ON DELETE CASCADE,
                      FOREIGN KEY (user_id) REFERENCES users (id) ON DELETE CASCADE
                    );""")

    connection.commit()

create_db()


def insert_user(username: str, password: str):
  with sqlite3.connect(db) as connection:
    cursor = connection.cursor()
    cursor.execute(f"""INSERT INTO users (username, password) VALUES ({username}, {password});""")
    connection.commit()

# insert_user('"Ivan"', '"12345"')
# insert_user('"Mike"', '"12345"')



def insert_chat(username: str, chatname: str):
  with sqlite3.connect(db) as connection:
    cursor = connection.cursor()
    cursor.execute(f"""INSERT INTO chats (chatname, user_id) VALUES ({chatname},
                      (SELECT id FROM users WHERE username = {username}));""")
    connection.commit()

# insert_chat('"Ivan"', '"ivans_chat"')
# insert_chat('"Mike"', '"ivans_chat"')
# insert_chat('"Ivan"', '"mikes_chat"')
# insert_chat('"Mike"', '"mikes_chat"')



def insert_message(message: str, username: str, chatname: str):
  with sqlite3.connect(db) as connection:
    cursor = connection.cursor()
    cursor.execute(f"""INSERT INTO messages (chat_id, user_id, message, created_at)
                      SELECT chats.id, users.id,{message}, CURRENT_TIMESTAMP
                      FROM users
                      JOIN chats ON chats.user_id = users.id
                      WHERE users.username={username} AND chats.chatname={chatname};""")
    connection.commit()



# insert_message('"hello"', '"Ivan"', '"ivans_chat"')
# insert_message('"hello"', '"Mike"', '"ivans_chat"')
# insert_message('"hello"', '"Ivan"', '"mikes_chat"')
# insert_message('"hello"', '"Mike"', '"mikes_chat"')


def delete(username: str, chatname: str):
  with sqlite3.connect(db) as connection:
    cursor = connection.cursor()
    cursor.execute(f"""DELETE FROM messages
                      WHERE chat_id = (
                      SELECT chats.id
                      FROM chats
                      JOIN users ON chats.user_id = users.id
                      WHERE users.username={username} AND chats.chatname={chatname});""")
    connection.commit()

delete('"Ivan"', '"new_chat"')